In [ ]:
# python version
!python -V

In [ ]:
# install/import the dependencies
!pip install -U ydata_profiling catboost -q

from typing import Tuple, List, Dict, Any
import pickle
import yaml
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from yaml import SafeLoader
from ydata_profiling import ProfileReport
from plotly.subplots import make_subplots
from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KernelDensity
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import (
    matthews_corrcoef, 
    roc_curve, 
    roc_auc_score, 
    classification_report
)

import warnings
warnings.filterwarnings("ignore")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.9/345.9 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.7/33.7 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.5/679.5 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 4.8 MB/s eta 0:00:00


In [ ]:
# DataFrame display options
pd.set_option('display.max_rows', 100) 
pd.set_option('display.max_columns', None) 

In [ ]:
# set sklearn's display configuration
set_config(display="diagram")

In [ ]:
# connect to the local file system 
from google.colab import files

In [ ]:
# upload the file(s) of interest
_ = files.upload()

In [ ]:
# confirm that the file(s) have been uploaded
!ls -lh

total 2.5M
-rw-r--r-- 1 root root 2.1M Apr 29 03:21 catboost_clf
-rw-r--r-- 1 root root  39K Apr 29 03:21 imputer.pkl
-rw-r--r-- 1 root root 356K Apr 29 03:21 income_evaluation.parquet
-rw-r--r-- 1 root root  587 Apr 29 03:20 parameters.yml
drwxr-xr-x 1 root root 4.0K Apr 27 13:35 sample_data


In [ ]:
# utils.py
def load_artifact(path: str) -> Any:
    """
    Reads in an artifact from path as a Python object

    Args:
        path: The artifact's file path

    Returns:
        obj: Python object
    """
    if path.split(".")[-1] == "parquet":
        obj = pd.read_parquet(path)
    elif path.split(".")[-1] == "yml":
        obj = yaml.load(open(path), Loader=SafeLoader)
    elif path.split(".")[-1] == "pkl":
        obj = pickle.load(open(path, "rb"))
    elif "catboost_clf" in path:
        obj = CatBoostClassifier().load_model(path)
    return obj

In [ ]:
# read in 'parameters.yml' as a dictionary
parameters = load_artifact(r"./parameters.yml")
parameters

{'target': 'income',
 'test_size': 0.2,
 'random_state': 42,
 'numeric_features': ['age', 'fnlwgt', 'hours_per_week'],
 'nominal_features': ['workclass',
  'marital_status',
  'occupation',
  'relationship',
  'race',
  'gender',
  'capital_gain',
  'capital_loss',
  'native_country'],
 'ordinal': {'features': ['education'],
  'categories': {'education': ['Preschool',
    '1st-4th',
    '5th-6th',
    '7th-8th',
    '9th',
    '10th',
    '11th',
    '12th',
    'HS-grad',
    'Some-college',
    'Assoc-voc',
    'Assoc-acdm',
    'Bachelors',
    'Masters',
    'Prof-school',
    'Doctorate']}}}

In [ ]:
# utils.py
def remove_whitespace(data: pd.DataFrame) -> pd.DataFrame:
    """
    Removes all white spaces from each categorical 
    column's entries
    """
    for col in data.select_dtypes("O").columns:
        data[col] = data[col].str.replace(" ", "")
    return data

In [ ]:
# utils.py
def preprocess_data(path: str) -> pd.DataFrame:
    """
    Returns a pre-processed DataFrame

    Args:
        path: Raw data's file path
    
    Returns:
        df: Pre-processed DataFrame
    """
    data = load_artifact(path)
    data.columns = [
        col.replace(" ", "").replace("-", "_").replace("sex", "gender") 
        for col in data.columns
    ]
    df = (
        data
        .pipe(remove_whitespace)
        .replace("?", np.nan)
        .assign(
            capital_gain=data["capital_gain"].apply(lambda x: "No" if x == 0 else "Yes").astype("object"), 
            capital_loss=data["capital_loss"].apply(lambda x: "No" if x == 0 else "Yes").astype("object")
        )
        .drop(["education_num"], axis=1)
        .copy(deep=True)
    )
    return df

In [ ]:
df = preprocess_data(r"./income_evaluation.parquet")

In [ ]:
# confirm that the 'df' DataFrame contains data
assert df.shape[0] > 0
assert df.shape[1] > 0

In [ ]:
# output the 1st five records from the 'df' DataFrame
df.head()

,age,workclass,fnlwgt,education,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income
0,39,State-gov,77516,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,Yes,No,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,No,No,13,United-States,<=50K
2,38,Private,215646,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,No,No,40,United-States,<=50K
3,53,Private,234721,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,No,No,40,United-States,<=50K
4,28,Private,338409,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,No,No,40,Cuba,<=50K


In [ ]:
# data type, percentage of missing values, and number of unique entries for each column
df_info = (pd.DataFrame(
    {
        "data_type": [str(dtype) for dtype in df.dtypes], 
        "percent_missing": [int(np.round(value)) for value in ((df.isna().sum() / df.shape[0]) * 100)], 
        "n_unique": df.nunique()
    }
    ).sort_values(["data_type", "percent_missing"], ascending=False)
)

df_info

,data_type,percent_missing,n_unique
workclass,object,6,8
occupation,object,6,14
native_country,object,2,41
education,object,0,16
marital_status,object,0,7
relationship,object,0,6
race,object,0,5
gender,object,0,2
capital_gain,object,0,2
capital_loss,object,0,2


In [ ]:
# create a ProfileReport instance for the 'df' DataFrame
profile = ProfileReport(df, explorative=True)

In [ ]:
# output the ProfileReport instance
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# save the ProfileReport instance as 'profile_report.html'
profile.to_file(r"./profile_report.html")

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

**`Feature imputation`**

In [ ]:
# utils.py 
def impute_features(
    data: pd.DataFrame, 
    params: Dict[str, Any]
) -> Tuple[pd.DataFrame, pd.DataFrame]: 
    """
    Imputes the train and test features that have missing values

    Args:
        data: Data containing missing values

    Return:
        imputed_train_set: Train set with imputed values
        imputed_test_set: Test set with imputed values
    """
    target = params["target"]
    train_set, test_set = train_test_split(
        data, 
        test_size=params["test_size"], 
        stratify=data[target], 
        shuffle=True
    )
    X_train = train_set.drop(target, axis=1).copy(deep=True)
    y_train = train_set[target].copy(deep=True)
    X_test = test_set.drop(target, axis=1).copy(deep=True)
    y_test = test_set[target].copy(deep=True)

    # a list of features that have missing values
    null_cols = [
        col
        for col in X_train.columns 
        if X_train[col].isna().sum() > 0
    ]

    # label encode the categorical features
    ctoi, itoc = {}, {}
    cat_cols = X_train.select_dtypes("O").columns.tolist()
    for col in cat_cols:
      categories = sorted(set(X_train[col].dropna()))
      indices = range(len(categories))
      ctoi[col] = dict(zip(categories, indices))
      itoc[col] = dict(zip(indices, categories))
      X_train[col] = X_train[col].map(ctoi[col])
      X_test[col] = X_test[col].map(ctoi[col])

    # # create an IterativeImputer instance and specify its arguments
    # imputer = IterativeImputer(
    #     estimator=DecisionTreeRegressor(
    #         splitter="best", 
    #         max_depth=3
    #     ), 
    #     imputation_order="ascending", 
    #     max_iter=10, 
    #     min_value=[X_train[col].min() for col in X_train.columns], 
    #     max_value=[X_train[col].max() for col in X_train.columns], 
    #     tol=1e-5/X_train.max().max(), 
    # )
    
    # # fit the 'imputer' object to the train set features
    # imputer.fit(X_train)

    # # save the 'imputer' object to the current working directory
    # pickle.dump(imputer, open(r"./imputer.pkl", "wb"))

    # read in the 'imputer' object
    imputer = load_artifact(r"./imputer.pkl")
    
    # impute the train and test set features that have missing values
    X_train = pd.DataFrame(
        imputer.transform(X_train), 
        columns=X_train.columns.tolist(), 
        index=X_train.index.tolist()
    )
    X_test = pd.DataFrame(
        imputer.transform(X_test), 
        columns=X_test.columns.tolist(), 
        index=X_test.index.tolist()
    )

    # map each categorical feature's entries back to their original categories
    for col in cat_cols:
      if col in null_cols:
        X_train[col] = np.abs(X_train[col]).round().astype(int).map(itoc[col])
        X_test[col] = np.abs(X_test[col]).round().astype(int).map(itoc[col])
      else:
        X_train[col] = X_train[col].astype(int).map(itoc[col])
        X_test[col] = X_test[col].astype(int).map(itoc[col])

    imputed_train_set = pd.concat([X_train, y_train], axis=1)
    imputed_test_set = pd.concat([X_test, y_test], axis=1)
    return imputed_train_set, imputed_test_set

In [ ]:
dftrain, dftest = impute_features(df, parameters)

In [ ]:
# confirm that all missing values have been imputed
assert dftrain.isna().sum().sum() == 0
assert dftest.isna().sum().sum() == 0

In [ ]:
# 1st five records from the 'dftrain' DataFrame
dftrain.head()

,age,workclass,fnlwgt,education,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income
18834,34.0,State-gov,221966.0,Bachelors,Never-married,Prof-specialty,Not-in-family,White,Male,No,No,45.0,United-States,<=50K
17854,36.0,Self-emp-not-inc,202950.0,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,No,No,50.0,Iran,<=50K
3299,19.0,Private,40425.0,Some-college,Never-married,Sales,Not-in-family,White,Female,No,No,28.0,United-States,<=50K
8357,39.0,Local-gov,134367.0,HS-grad,Never-married,Adm-clerical,Own-child,White,Female,No,No,35.0,United-States,<=50K
23810,43.0,Local-gov,227734.0,Assoc-voc,Married-civ-spouse,Transport-moving,Husband,White,Male,No,No,22.0,United-States,<=50K


In [ ]:
# 1st five records from the 'dftest' DataFrame
dftest.head()

,age,workclass,fnlwgt,education,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income
22967,25.0,Private,256620.0,Bachelors,Separated,Exec-managerial,Not-in-family,White,Male,No,No,40.0,United-States,<=50K
1836,58.0,Private,312131.0,Masters,Married-civ-spouse,Sales,Husband,White,Male,No,No,40.0,United-States,<=50K
27267,62.0,Private,181014.0,Bachelors,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,No,No,40.0,United-States,<=50K
21045,36.0,Self-emp-not-inc,37778.0,Masters,Married-civ-spouse,Farming-fishing,Husband,White,Male,No,No,70.0,United-States,<=50K
27815,39.0,Private,225544.0,12th,Never-married,Handlers-cleaners,Not-in-family,White,Male,No,No,40.0,United-States,<=50K


**`Numeric feature outlier analysis`**

In [ ]:
# boxplot for each numeric feature to identify their outliers
target = parameters["target"]
df_imputed = pd.concat([dftrain, dftest], axis=0).sort_index(axis=0)
num_cols = df_imputed.drop(target, axis=1).select_dtypes("number").columns.tolist()
fig = make_subplots(rows=1, cols=len(num_cols))
for i, col in enumerate(sorted(num_cols)):
    fig.add_trace(
        go.Box(y=df_imputed[col], name=col.title(), boxpoints="outliers"), 
        row=1, 
        col=i+1
    )

fig.show()

In [ ]:
# output the percentage of outliers for each numeric feature
outliers = {}
for col in sorted(num_cols):
    iqr = df_imputed[col].quantile(0.75) - df_imputed[col].quantile(0.25)
    lower_threshold = df_imputed[col].quantile(0.25) - (1.5 * iqr)
    upper_threshold = df_imputed[col].quantile(0.75) + (1.5 * iqr)
    df_outliers = (
        df_imputed
        [(df_imputed[col] < lower_threshold) | 
         (df_imputed[col] > upper_threshold)
        ]
    )
    percent_outliers = np.round(df_outliers.shape[0] / df_imputed.shape[0], 4)
    outliers[col] = percent_outliers

outliers

{'age': 0.0044, 'fnlwgt': 0.0305, 'hours_per_week': 0.2766}

**`Exploratory data analysis`**

In [ ]:
# output the distribution of each numeric feature
target = parameters["target"]
num_cols = df_imputed.drop(target, axis=1).select_dtypes("number").columns

fig = make_subplots(rows=1, cols=3)
for i, col in enumerate(sorted(num_cols)):
    df_eda = df_imputed[[col, target]]
    df_pos = df_eda[df_eda[target] == ">50K"]
    df_neg = df_eda[df_eda[target] == "<=50K"]
    pos_trace = go.Histogram(
        x=df_pos[col], 
        nbinsx=int(np.round(1 + (3.322 * np.log(df_pos.shape[0])))), # Sturge's Rule for optimizing the number of histogram bins
        marker_color="green",
        name=f"{col.title()}>50K"
    )
    fig.append_trace(pos_trace, row=1, col=i+1)
    neg_trace = go.Histogram(
        x=df_neg[col], 
        nbinsx=int(np.round(1 + (3.322 * np.log(df_neg.shape[0])))), 
        marker_color="red", 
        name=f"{col.title()}<=50K"
    )
    fig.append_trace(neg_trace, row=1, col=i+1)
    fig.update_xaxes(title_text=col.title(), row=1, col=i+1)
    fig.update_yaxes(title_text="Count", row=1, col=i+1)
    
fig.update_layout(
    title_text=f"Numeric feature distributions, color coded by {target}", 
    legend_title=target, 
    barmode="stack", 
    showlegend=False
)
fig.update_traces(opacity=0.5)
fig.show()

In [ ]:
# save the above figure as 'numeric_features_distribution.html'
fig.write_html(r"./numeric_features_distribution.html")

In [ ]:
# create a histogram for each numeric feature
target = parameters["target"]
num_cols = df_imputed.drop(target, axis=1).select_dtypes("number").columns
fig = make_subplots(rows=len(num_cols), cols=1)
for i, col in enumerate(sorted(num_cols)):
    df_eda = (
        df_imputed
        .groupby([col, target])
        .size()
        .reset_index(name="count")
    )
    fig = px.histogram(
        df_eda, 
        x=col, 
        y="count", 
        color=target, 
        marginal="rug", 
        color_discrete_map={">50K": "green", "<=50K": "red"}, 
        title=f"Distribution of {col}, color coded by income"
    )
    fig.show()

In [ ]:
# how are gender and race related to income?
target = parameters["target"]
df_eda = (
    df_imputed
    .groupby(["gender", "race", target])
    .size()
    .reset_index(name="count")
)

# normalize the 'count' feature
df_eda["normalized_count"] = df_eda["count"] / df_imputed.shape[0]

# output five random records from the 'df_eda' DataFrame
df_eda.sample(5)

,gender,race,income,count,normalized_count
3,Female,Asian-Pac-Islander,>50K,43,0.001321
17,Male,Other,>50K,19,0.000584
7,Female,Other,>50K,6,0.000184
9,Female,White,>50K,1028,0.031572
5,Female,Black,>50K,90,0.002764


In [ ]:
# create a treemap chart for the above DataFrame
fig = px.treemap(
    df_eda, 
    path=df_eda.drop("count", axis=1).columns[:-1], 
    values=df_eda.columns[-1], 
    color=target, 
    title="How are gender and race related to income?"
)

fig.show()

In [ ]:
# save the above figure as 'gender_race_income.html'
fig.write_html(r"./gender_race_income_treemap.html")

In [ ]:
# how is income related to gender and education?
df_eda = (
    df_imputed
    .groupby([target, "gender", "education"])
    .size()
    .reset_index(name="count")
)

# normalize the 'count' feature
df_eda["normalized_count"] = df_eda["count"] / df.shape[0]

# output five random records from the 'df_eda' DataFrame
df_eda.sample(5)

,income,gender,education,count,normalized_count
20,<=50K,Male,5th-6th,235,0.007217
49,>50K,Male,1st-4th,6,0.000184
60,>50K,Male,Some-college,1190,0.036547
38,>50K,Female,Assoc-acdm,56,0.001720
22,<=50K,Male,9th,348,0.010688


In [ ]:
# create a sunburst plot for the above DataFrame
fig = px.sunburst(
    df_eda, 
    path=df_eda.drop("count", axis=1).columns[:-1], 
    values=df_eda.columns[-1], 
    title="How is income related to gender and education?"
)

fig.show()

In [ ]:
# save the above figure as 'income_gender_education_sunburstchart.html'
fig.write_html(r"./income_gender_education_sunburstchart.html")

In [ ]:
# how is income related to capital gains and occupation?
df_eda = (
    df_imputed
    .groupby(["capital_gain", target, "occupation"])
    .size()
    .reset_index(name="count")
)

# normalize the 'count' feature
df_eda["normalized_count"] = df_eda["count"] / df.shape[0]

# output five random records from the 'df_eda' DataFrame
df_eda.sample(5)

,capital_gain,income,occupation,count,normalized_count
5,No,<=50K,Handlers-cleaners,1693,0.051995
16,No,>50K,Craft-repair,755,0.023187
30,Yes,<=50K,Farming-fishing,43,0.001321
24,No,>50K,Sales,769,0.023617
32,Yes,<=50K,Machine-op-inspct,135,0.004146


In [ ]:
# how are capital gains related to income and occupation?
fig = px.sunburst(
    df_eda, 
    path=df_eda.drop("count", axis=1).columns[:-1], 
    values=df_eda.columns[-1], 
    title="How are capital gains related to income and occupation?"
)

fig.show()

In [ ]:
# save the above figure as 'capgains_income_occupation_sunburstchart.html'
fig.write_html(r"./capgains_income_occupation_sunburstchart.html")

**`Feature transformation`**

In [ ]:
# utils.py
def transform_features(
    train_set: pd.DataFrame, 
    test_set: pd.DataFrame, 
    params: Dict[str, Any]
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.Series, pd.Series]: 
    """
    Returns ML-ready train and test set features and targets
    
    Args:
        train_set: Train set with no missing values  
        test_set: Test set with no missing values
        params: Parameters defined parameters.yml 

    Returns:
        X_train: ML-ready train set feature matrix
        X_test: ML-ready test set features matrix
        y_train: Train set target vector
        y_test: Test set target vector
    """
    target = params["target"]
    target_classes = sorted(set(train_set[target]))
    target_encoder = dict(zip(target_classes, [0, 1]))
    df_train, df_test = train_set.copy(deep=True), test_set.copy(deep=True)
    for df in [df_train, df_test]:
        df[target] = df[target].map(target_encoder)
    
    numeric_cols = params["numeric_features"]
    nominal_cols = params["nominal_features"] 
    ordinal_cols = params["ordinal"]["features"]
    for col in numeric_cols + nominal_cols + ordinal_cols:
        if col in numeric_cols:
            train_mean, train_std = df_train[col].mean(), df_train[col].std()
            df_train[col] = (df_train[col] - train_mean) / train_std
            df_test[col] = (df_test[col] - train_mean) / train_std
        elif col in nominal_cols:
            categories = df_train.groupby(col)[target].mean().index.tolist()
            values = df_train.groupby(col)[target].mean().values.tolist()
            ctov = dict(zip(categories, values))
            df_train[col] = df_train[col].map(ctov)
            df_test[col] = df_test[col].map(ctov)
            train_mean, train_std = df_train[col].mean(), df_train[col].std()
            df_train[col] = (df_train[col] - train_mean) / train_std
            df_test[col] = (df_test[col] - train_mean) / train_std
        else:
            categories = params["ordinal"]["categories"][col]
            indices = range(len(categories))
            ctoi = dict(zip(categories, indices))
            df_train[col] = df_train[col].map(ctoi)
            df_test[col] = df_test[col].map(ctoi)
            train_mean, train_std = df_train[col].mean(), df_train[col].std()
            df_train[col] = (df_train[col] - train_mean) / train_std
            df_test[col] = (df_test[col] - train_mean) / train_std
    
    X_train, y_train = df_train.drop(target, axis=1), df_train[target]
    X_test, y_test = df_test.drop(target, axis=1), df_test[target]
    return X_train, X_test, y_train, y_test

In [ ]:
Xtrain, Xtest, ytrain, ytest = transform_features(dftrain, dftest, parameters)

In [ ]:
# confirm that all train and test features are numeric
assert ", ".join(Xtrain.columns.tolist()) == ", ".join(Xtrain.select_dtypes("number").columns.tolist())
assert ", ".join(Xtest.columns.tolist()) == ", ".join(Xtest.select_dtypes("number").columns.tolist())

In [ ]:
# 1st five records from the 'Xtrain' DataFrame
Xtrain.head()

,age,workclass,fnlwgt,education,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country
18834,-0.338625,0.588378,0.308140,1.126055,-1.016396,1.376289,-0.706501,0.344106,0.708091,-0.303873,-0.223093,0.376190,0.112880
17854,-0.191766,0.605322,0.126884,1.126055,1.079082,1.602312,1.070774,0.344106,0.708091,-0.303873,-0.223093,0.781923,3.324675
3299,-1.440068,-0.460137,-1.422263,-0.032865,-1.016396,0.212998,-0.706501,0.344106,-1.412194,-0.303873,-0.223093,-1.003300,0.112880
8357,0.028523,0.704656,-0.526832,-0.419171,-1.016396,-0.693677,-1.166456,0.344106,-1.412194,-0.303873,-0.223093,-0.435275,0.112880
23810,0.322241,0.704656,0.363119,0.353442,1.079082,-0.302170,1.070774,0.344106,0.708091,-0.303873,-0.223093,-1.490179,0.112880


In [ ]:
# 1st five records from the 'Xtest' DataFrame
Xtest.head()

,age,workclass,fnlwgt,education,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country
22967,-0.999491,-0.460137,0.638453,1.126055,-0.901154,1.602312,-0.706501,0.344106,0.708091,-0.303873,-0.223093,-0.029542,0.11288
1836,1.423684,-0.460137,1.167570,1.512361,1.079082,0.212998,1.070774,0.344106,0.708091,-0.303873,-0.223093,-0.029542,0.11288
27267,1.717402,-0.460137,-0.082204,1.126055,1.079082,-0.852357,1.070774,0.344106,0.708091,-0.303873,-0.223093,-0.029542,0.11288
21045,-0.191766,0.605322,-1.447494,1.512361,1.079082,-0.879617,1.070774,0.344106,0.708091,-0.303873,-0.223093,2.404853,0.11288
27815,0.028523,-0.460137,0.342244,-0.805478,-1.016396,-1.118256,-0.706501,0.344106,0.708091,-0.303873,-0.223093,-0.029542,0.11288


**`Model training and evaluation`**

In [ ]:
def evaluate_models(data: pd.DataFrame, params: Dict[str, Any]) -> Dict[str, list]:
    """
    Returns a dictionary that maps each classifier to its test set AUC score

    Args:
        data: Raw data
        params: Parameters defined in 'parameters.yml'
    
    Returns:
        report: Dictionary that maps each classifier to its test set AUC score
    """
    models = {
        "LogisticRegression": LogisticRegression(
            solver="newton-cholesky", 
            max_iter=3000
        ), 
        "XGBClassifier": XGBClassifier(objective="binary:logistic"), 
        "LGBMClassifier": LGBMClassifier(
            boosting_type="gbdt", 
            objective="binary", 
            metric="auc"
        ), 
        "CatBoostClassifier": CatBoostClassifier(
            eval_metric="AUC", 
            verbose=False
        ) 
    }
    target: str = params["target"]
    cat_cols = params["nominal_features"] + params["ordinal"]["features"]
    train_set, test_set = impute_features(data, params)
    report = {}
    for name, clf in models.items():
        if isinstance(clf, type(LGBMClassifier())):
            X_train = train_set.drop(target, axis=1).copy(deep=True)
            y_train = train_set[target].copy(deep=True)
            X_test = test_set.drop(target, axis=1).copy(deep=True)
            y_test = test_set[target].copy(deep=True)
            for col in cat_cols:
                X_train[col] = X_train[col].astype("category")
                X_test[col] = X_test[col].astype("category")
            clf.fit(X_train, y_train)
            pos_class_pred_proba = clf.predict_proba(X_test)[:, 1]
            test_auc = roc_auc_score(y_test, pos_class_pred_proba)
            report[name] = [clf, test_auc.round(4)]
        elif isinstance(clf, type(CatBoostClassifier())):
            pool_train = Pool(
                train_set.drop(target, axis=1).copy(deep=True), 
                train_set[target].copy(deep=True), 
                cat_features=cat_cols
            )
            pool_test = Pool(
                test_set.drop(target, axis=1).copy(deep=True), 
                test_set[target].copy(deep=True), 
                cat_features=cat_cols
            )
            clf.fit(pool_train)
            pos_class_pred_proba = clf.predict_proba(pool_test)[:, 1]
            test_auc = roc_auc_score(test_set[target], pos_class_pred_proba)
            report[name] = [clf, test_auc.round(4)]
        else:
            X_train, X_test, y_train, y_test = transform_features(
                train_set, test_set, params
            )
            clf.fit(X_train, y_train)
            pos_class_pred_proba = clf.predict_proba(X_test)[:, 1]
            test_auc = roc_auc_score(y_test, pos_class_pred_proba)
            report[name] = [clf, test_auc.round(4)]

    report = dict(sorted(report.items(), key=lambda x: x[1][1])[::-1])
    return report

In [ ]:
report = evaluate_models(df, parameters)
report

{'CatBoostClassifier': [<catboost.core.CatBoostClassifier at 0x7fba878d70a0>,
  0.9127],
 'LGBMClassifier': [LGBMClassifier(metric='auc', objective='binary'), 0.9103],
 'XGBClassifier': [XGBClassifier(base_score=None, booster=None, callbacks=None,
                colsample_bylevel=None, colsample_bynode=None,
                colsample_bytree=None, early_stopping_rounds=None,
                enable_categorical=False, eval_metric=None, feature_types=None,
                gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
                interaction_constraints=None, learning_rate=None, max_bin=None,
                max_cat_threshold=None, max_cat_to_onehot=None,
                max_delta_step=None, max_depth=None, max_leaves=None,
                min_child_weight=None, missing=nan, monotone_constraints=None,
                n_estimators=100, n_jobs=None, num_parallel_tree=None,
                predictor=None, random_state=None, ...),
  0.9031],
 'LogisticRegression': [Logis

In [ ]:
# extract the classifier that produced the highest test set AUC score
clf = report["CatBoostClassifier"][0]
clf

In [ ]:
# baseline: assume that all the target labels belong to the negative class, i.e., "<=50K"
target = parameters["target"]
base_ytrain = pd.Series(["<=50K"] * (dftrain.shape[0] - 1) + [">50K"], index=dftrain.index.tolist())
base_train = Pool(
    dftrain.drop(target, axis=1).copy(deep=True), 
    base_ytrain, 
    cat_features=dftrain.drop(target, axis=1).select_dtypes("O").columns.tolist()
)

base_ytest = pd.Series(["<=50K"] * dftest.shape[0], index=dftest.index.tolist())
base_test = Pool(
    dftest.drop(target, axis=1).copy(deep=True), 
    base_ytest, 
    cat_features=dftrain.drop(target, axis=1).select_dtypes("O").columns.tolist()
)

base_clf = CatBoostClassifier(eval_metric="AUC", verbose=False)
base_clf.fit(base_train)

In [ ]:
# output the test set's baseline metrics
pos_class_pred_proba = base_clf.predict_proba(base_test)[:, 1]
base_auc = roc_auc_score(dftest[target], pos_class_pred_proba) 
base_acc = np.mean(dftest[target].values == base_ytest.values)
base_mccor = matthews_corrcoef(dftest[target].values, base_ytest.values)
print(f"Baseline AUC Score: {base_auc:.2f}; Baseline Accuracy: {base_acc:.2f}; Baseline Matthew's Correlation: {base_mccor:.2f}")

Baseline AUC Score: 0.50; Baseline Accuracy: 0.76; Baseline Matthew's Correlation: 0.00


In [ ]:
# output the test set's updated metrics
test_auc = report["CatBoostClassifier"][1]
test_acc = np.mean(dftest[target].values == clf.predict(dftest.drop(target, axis=1)))
test_mccor = matthews_corrcoef(dftest[target].values, clf.predict(dftest.drop(target, axis=1)))
print(f"Updated AUC Score: {test_auc:.2f}; Updated Accuracy: {test_acc:.2f}; Updated Matthew's Correlation: {test_mccor:.2f}")

Updated AUC Score: 0.92; Updated Accuracy: 0.87; Updated Matthew's Correlation: 0.61


In [ ]:
# save the 'clf' object as 'catboost_clf'
clf.save_model(r"./catboost_clf")

**`Predicting a single record`**

In [ ]:
# read in 'catboost_clf'
model = load_artifact(r"./catboost_clf")

In [ ]:
# output a single record from the 'dftest' DataFrame
target = parameters["target"]
idx = np.random.choice(dftest.index.tolist())
record = dftest.loc[idx, :].to_frame().T
x = record.drop(target, axis=1).copy(deep=True)
x

,age,workclass,fnlwgt,education,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country
8,31.0,Private,45781.0,Masters,Never-married,Prof-specialty,Not-in-family,White,Female,Yes,No,50.0,United-States


In [ ]:
# output the corresponding label and prediction
y = record[target].values[0]
yhat = model.predict(x)[0]
print(f"Actual label: {y}; Predicted label: {yhat}")

Actual label: >50K; Predicted label: >50K


**`Bayesian hyperparameter optimization: XGBoost example`**

In [ ]:
# create a DataFrame containing 20 initial hyperparameter samples
# note, each row represents a hyperparameter sample
initial_samples = pd.DataFrame(
    {
        "n_estimators": np.random.randint(low=100, high=1000, size=20), 
        "max_depth": np.random.randint(low=1, high=20, size=20),
        "eta": np.random.uniform(low=0, high=1, size=20), 
        "gamma": np.clip(np.random.exponential(scale=50, size=20), 0, 100), 
        "min_child_weight": np.clip(np.random.exponential(scale=10, size=20), 0, 30), 
        "max_delta_step": np.clip(np.random.exponential(scale=10, size=20), 0, 30), 
        "subsample": np.random.uniform(low=0, high=1, size=20), 
        "lambda": np.clip(np.random.exponential(scale=5, size=20), 0, 20), 
        "alpha": np.clip(np.random.exponential(scale=5, size=20), 0, 20)
    }
)

# output five random hyperparameter samples
initial_samples.sample(5)

In [ ]:
# extract a random hyperparameter sample as a dictionary
idx = np.random.choice(initial_samples.index.tolist())
random_sample = initial_samples.iloc[idx, :].to_dict()

# convert the 'n_estimators' and 'max_depth' values to integers
random_sample["n_estimators"] = int(random_sample["n_estimators"])
random_sample["max_depth"] = int(random_sample["max_depth"])

# output the random hyperparameter sample
random_sample

In [ ]:
def score_sample(
    sample: Dict[str, np.number], 
    X_train: pd.DataFrame, 
    X_test: pd.DataFrame, 
    y_train: pd.Series, 
    y_test: pd.Series
) -> float:
    """
    Returns the model's test set evaluation 
    metric, based on its set of hyperparameters

    Args:
        sample: Set of hyperparameters
        X_train: ML-ready train set feature matrix
        X_test: ML-ready test set feature matrix
        y_train: ML-ready train set target vector
        y_test: ML-ready test set rarget vector

    Returns:
        auc_score: Test set's evaluation metric 
    """
    # instantiate an XGBClassifier
    clf = XGBClassifier(
        objective="binary:logistic", 
        eval_metric="auc", 
        early_stopping_rounds=10, 
        **sample
    )
    
    # train the classifier
    clf.fit(
        X_train, 
        y_train, 
        eval_set=[(X_test, y_test)], 
        verbose=0
    )

    # test set positive class prediction probabilities
    pos_class_pred_proba = clf.predict_proba(X_test)[:, 1]

    # evaluate the classifier via the AUC score
    auc_score = roc_auc_score(y_test, pos_class_pred_proba)

    return auc_score

In [ ]:
random_score = score_sample(random_sample, Xtrain, Xtest, ytrain, ytest)
random_score

In [ ]:
def get_initial_scores(
    samples: pd.DataFrame, 
    data: pd.DataFrame, 
    params: Dict[str, Any]
) -> List[float]:
    """
    Returns a list of test set evaluation 
    metrics, one for each set of hyperparameters
    
    Args:
        samples: Collection of initial hyperparameter sets (priors)
        data: Raw data
        params: Parameters defined in 'parameters.yml'

    Returns:
        auc_sccores: List of test set evaluation
        metrics, one per hyperparameter set
    """
    auc_scores = []
    for i in range(samples.shape[0]):
        sample = samples.iloc[i, :].to_dict()
        sample["n_estimators"] = int(sample["n_estimators"])
        sample["max_depth"] = int(sample["max_depth"])
        train_set, test_set = impute_features(data, params)
        X_train, X_test, y_train, y_test = transform_features(
            train_set, 
            test_set, 
            params
        )
        auc_score = score_sample(
            sample, 
            X_train, 
            X_test, 
            y_train, 
            y_test
        )
        auc_scores.append(auc_score)
    return auc_scores

In [ ]:
auc_scores = get_initial_scores(initial_samples, df, parameters)

In [ ]:
# extract a threshold auc score that corresponds to the 90th percentile...
# of all the scores in the 'auc_scores' list
alpha = np.quantile(auc_scores, 0.9)
alpha

In [ ]:
# output the hyperparameter samples that produced an auc score greater than the threshold
initial_samples[auc_scores >= alpha]

In [ ]:
def split_samples(
    samples: pd.DataFrame, 
    auc_scores: List[float], 
    quantile=0.9
) -> Tuple[Any, Any]:
    """
    Returns two multivariate kernel density estimators, one representing 
    the optimal hyperparameter sets, and the other representing the 
    sub-optimal hyperparameter sets

    Args:
        samples: Collection of initial hyperparameter sets (priors)
        auc_scores: List of test set evaluation metrics, one for each 
        hyperparameter set in samples
        quantile: Threshold that differentiates the optimal hyperparameter 
        sets from the sub-optimal hyperparameter sets
    
    Returns:
        good_kde: Multivariate kernel density estimator for the optimal 
        hyperparameter sets
        bad_kde: Multivariate kernel density estimator for the sub-optimal 
        hyperparameter sets
    """
    # threshold auc score that differentiates the 'good' and 'bad' hyperparameter sets
    alpha = np.quantile(auc_scores, quantile)

    # fit a kernel density estimator to the 'good' hyperparameter sets
    good_samples = samples[auc_scores >= alpha].copy(deep=True)
    good_kde = KernelDensity(kernel="gaussian", bandwidth=0.1)
    good_kde.fit(good_samples)

    
    # fit a kernel density estimator to the 'bad' hyperparameter samples
    bad_samples = samples[auc_scores < alpha].copy(deep=True)
    bad_kde = KernelDensity(kernel="gaussian", bandwidth=0.1)
    bad_kde.fit(bad_samples)

    return good_kde, bad_kde

In [ ]:
kde_good, kde_bad = split_samples(initial_samples, auc_scores)

In [ ]:
def get_sample(
    samples: pd.DataFrame, 
    auc_scores: List[float], 
    n_samples=100
) -> Dict[str, np.number]:
    """
    Returns a hyperparameter set that's (A) sampled from the multivariate 
    distribution of optimal hyperparameters and (B) maximizes the likelihood 
    of optimal hyperparameters and minimizes the likelihood of sub-optimal 
    hyperparameters

    Args:
        samples: Collection of initial hyperparameter sets (priors)
        auc_scores: List of test set evaluation metrics, one for each 
        hyperparameter set in samples
        n_samples: The number of hyperparameter sets to be sampled from 
        the multivariate distribution of optimal hyperparameters
    
    Returns: 
        sample: The hyperparameters set that corresponds to the maximum 
        likelihood of optimal hyperparameters and minimum likelihood of 
        sub-optimal hyperparameters
    """
    # standardize the hyperparameter samples for easier kernel fitting
    hyperparam_means = samples.mean()
    hyperparam_stds = samples.std()
    standardized_samples = (samples - hyperparam_means) / hyperparam_stds
    
    # extract the 'good' and 'bad' kernel density estimators
    kde_good, kde_bad = split_samples(standardized_samples, auc_scores)

    # sample N hyperparameter samples from the 'good' kernel density estimator's distribution
    new_samples = pd.DataFrame(
        kde_good.sample(n_samples), 
        columns=standardized_samples.columns
    )

    # calculate the likelihoods w.r.t. the 'good and 'bad' kernel density estimators
    likelihoods_good = np.exp(kde_good.score_samples(new_samples))
    likelihoods_bad = np.exp(kde_bad.score_samples(new_samples))

    # extract the index that corresponds to the hyperparameter sample that...
    # minimizes likelihoods_good / likelihoods_bad
    idx = np.argmax(likelihoods_good / likelihoods_bad)

    # extract the 'best' hyperparameter sample from the 'new_samples' DataFrame
    sample = new_samples.iloc[idx, :]

    # undo standardization and convert the sample from a Series to a dictionary
    sample = (sample * hyperparam_stds + hyperparam_means).to_dict()
    
    # ensure all hyperparameters are within their given ranges
    sample["n_estimators"] = np.clip(int(sample["n_estimators"]), 100, 1000)
    sample["max_depth"] = np.clip(int(sample["max_depth"]), 1, 20)
    sample["eta"] = np.clip(sample["eta"], 0, 1)
    sample["gamma"] = np.clip(sample["gamma"], 0 , 100)
    sample["min_child_weight"] = np.clip(sample["min_child_weight"], 0, 30)
    sample["max_delta_step"] = np.clip(sample["max_delta_step"], 0, 30)
    sample["subsample"] = np.clip(sample["subsample"], 0, 1)
    sample["lambda"] = np.clip(sample["lambda"], 0, 20)
    sample["alpha"] = np.clip(sample["alpha"], 0, 20)
    return sample

In [ ]:
get_sample(initial_samples, auc_scores)

In [ ]:
# a dictionary to map each AUC score to its corresponding hyperparameter set
score_to_sample = {}

# a list to store the final AUC scores
final_scores = []

iteration = 0
while iteration < 100:
    initial_scores: list = get_initial_scores(initial_samples, df, parameters)
    best_sample: dict = get_sample(initial_samples, initial_scores)
    best_score: float = score_sample(best_sample, Xtrain, Xtest, ytrain, ytest)
    score_to_sample[str(best_score)] = best_sample
    final_scores.append(best_score)
    
    iteration += 1

In [ ]:
plt.plot(final_scores, 'o');

In [ ]:
# sort the AUC scores the 'final_scores' list in descending order 
final_scores.sort(reverse=True)

# extract the 'best' AUC score
score = final_scores[0]

# extract hyperparameter set that corresponds to the 'best' AUC score
sample = score_to_sample[str(score)]

# output the hyperparameter set that corresponds to the 'best' AUC score
print(f"Best AUC score: {score.round(4)}")
print("Best hyperparameter set:")
sample

In [ ]:
# extract the train and test set features and targets
dftrain, dftest = impute_features(df, parameters)
Xtrain, Xtest, ytrain, ytest = transform_features(dftrain, dftest, parameters)

# create a model using the hyperparameter set from the above code cell
clf = XGBClassifier(
    objective="binary:logistic", 
    eval_metric="auc", 
    early_stopping_rounds=50, 
    **sample,
)

# train the model
clf.fit(
    Xtrain, 
    ytrain, 
    eval_set=[(Xtest, ytest)], 
    verbose=1
)

In [ ]:
# output the test set accuracy and Matthew's correlation coefficient
test_accuracy = np.mean(ytest.values == clf.predict(Xtest))
test_mcorr = matthews_corrcoef(ytest.values, clf.predict(Xtest))
print(f"Accuracy: {test_accuracy:.2f}; Matthew's Correlation Coefficient: {test_mcorr:.2f}")

In [ ]:
# save the model as 'model.pkl'
pickle.dump(clf, open(r"./model.pkl", "wb"))